In [19]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def relu_bn(inputs: Tensor) -> Tensor:

    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(128, 128, 3)) # Input(shape=(32, 32, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling2D()(t) # t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(2, activation='softmax')(t) # Dense(10, activation='softmax')
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy', #sparse_categorical_crossentropy
        metrics=['accuracy']
    )

    return model

In [3]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def create_plain_net():
    
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [4, 10, 10, 4]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            downsample = (j==0 and i!=0)
            t = Conv2D(kernel_size=3,
                       strides= (1 if not downsample else 2),
                       filters=num_filters,
                       padding="same")(t)
            t = relu_bn(t)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# kraschar ofta minnet för att den är för stor. Man kan minska batchen för att den inte ska krascha, men kraschar fortfarande vid test
# Det går inte på ett smidigt sätt spara
import h5py
import numpy as np
import os
import time
import datetime
start = time.time()

image_batch = None
label_batch = None
root = '/content/gdrive/MyDrive/wikiart_sample_test'

with h5py.File(os.path.join(root, 'wikiart_sample_test.hdf5'), 'r') as file:
    image_batch = np.array(file["/images"], dtype='float32') 
    label_batch = np.array(file["/meta"])

end = time.time()

print(f"It took : {datetime.timedelta(seconds=int(end - start))}")

In [6]:
# Använd för att läsa in bilder i ordningsföljd med specifika kategorier
# Bra om man vill hålla dessa variablar statiska, när man experimenterar
# ! Om man vill köra mindre två så behöver man göra förändringar enligt första versionen

import random 

nr_of_images_per_style_first = 2
sample_styles = ['Abstract_Expressionism', 'Contemporary_Realism'] 

root = "/content/gdrive/MyDrive/project-art/datasets/Wikiart-dataset/wikiart/"
images_labels = []

for folder_name in sample_styles:
  images = os.listdir(os.path.join(root, folder_name))[:nr_of_images_per_style_first]
  images_labels.extend([(image, folder_name) for image in images])

random.seed(42)
random.shuffle(images_labels)

In [7]:
import cv2
import time
import datetime
from tensorflow.keras.preprocessing.image import img_to_array
start = time.time()

image_batch = []
label_batch = [] 

for image, label in images_labels:
  image_path = os.path.join(root, label, image)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (128, 128))
  image = img_to_array(image)
  image_batch.append(image)
  label_batch.append(label)

end = time.time()

print(f"It took : {datetime.timedelta(seconds=int(end - start))}")

It took : 0:00:01


In [12]:
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
image_batch = np.array(image_batch, dtype="float32") / 255.0
label_batch = np.array(label_batch)
mlb = LabelBinarizer()
label_batch = mlb.fit_transform(label_batch)
from keras.utils import to_categorical

In [13]:
print(label_batch[0])

[1]


In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_batch,label_batch,test_size=0.5,random_state=42)

In [16]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

if None in x_train:
  print("ehh")
if None in x_train:
  print("ehh")
if None in x_test:
  print("ehh")
if None in y_test:
  print("ehh")

(2, 128, 128, 3)
(2, 1)
(2, 128, 128, 3)
(2, 1)


In [18]:
# from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os

# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

model = create_res_net() # or create_plain_net()
model.summary()

# timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# name = 'cifar-10_res_net_30-'+timestr # or 'cifar-10_plain_net_30-'+timestr

# checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
# cp_callback = ModelCheckpoint(
#    filepath=checkpoint_path,
#    verbose=1
#)
#tensorboard_callback = TensorBoard(
#    log_dir='tensorboard_logs/'+name,
#    histogram_freq=1
#)

checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

model.fit(
    x=x_train,
    y=y_train,
    epochs=1,
    verbose=1,
    validation_data=(x_test, y_test),
    batch_size=50,
    callbacks=[checkpoint] # cp_callback, tensorboard_callback
)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, 128, 128, 3)  12          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 128, 128, 64) 1792        batch_normalization_30[0][0]     
__________________________________________________________________________________________________
re_lu_29 (ReLU)                 (None, 128, 128, 64) 0           conv2d_32[0][0]                  
____________________________________________________________________________________________

ValueError: ignored

In [25]:
loss, accuracy = model.evaluate(x_test, y_test, len(y_test))

1/1 [==============================] - 2s 2s/step - loss: 0.7714 - accuracy: 0.0000e+00
